In [2]:

import requests
from tqdm import tqdm

url = 'https://www.dropbox.com/scl/fi/jsumicca8xqj9qwjaw8ac/test.tar?rlkey=m44x5olxh8v6jwu27n54nk796&st=ofto2v33&dl=1'
file_path = 'test.tar'

# Send a request to get the file size for progress tracking
response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))

# Download the file with a progress bar
with open(file_path, 'wb') as file, tqdm(
    desc="Downloading",
    total=total_size,
    unit='B',
    unit_scale=True,
    unit_divisor=1024,
) as progress_bar:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)
            progress_bar.update(len(chunk))

print("Download completed successfully.")


Downloading:   2%|▏         | 29.3M/1.72G [00:13<12:55, 2.35MB/s]


KeyboardInterrupt: 

In [4]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import shutil

def copy_images_and_masks(relation_folder, image_dir, mask_dir, augmented_dir, target_size=(128, 128), num_classes=21):
    collected_image_names = set()

    json_files = [f for f in os.listdir(relation_folder) if f.endswith('.json')]
    for json_file in tqdm(json_files, desc="Collecting image names from JSON files"):
        json_path = os.path.join(relation_folder, json_file)
        with open(json_path, 'r') as f:
            data = json.load(f)
            file_names = data.get("fileNames", [])
            for file_name in file_names:
                base_name = os.path.splitext(file_name)[0]
                collected_image_names.add(base_name)
    
    for base_name in tqdm(sorted(collected_image_names), desc="Loading images and masks from relations"):
        img_path = os.path.join(image_dir, f"{base_name}.jpg")
        img_path_out = './img'
        mask_path = os.path.join(mask_dir, f"{base_name}.png")
        mask_path_out = './msk'
        
        if os.path.exists(img_path):
            shutil.copy(img_path, img_path_out)
        
        if os.path.exists(mask_path):
            shutil.copy(mask_path, mask_path_out)
            
    
    augmented_files = os.listdir(augmented_dir)
    augmented_base_names = {f.split('-')[0] for f in augmented_files}
    
    for base_name in tqdm(sorted(augmented_base_names), desc="Loading augmented images and masks"):
        img_files = [f for f in augmented_files if f.startswith(base_name)]
        for img_file in img_files:
            aug_img_path = os.path.join(augmented_dir, img_file)
            img_path_out = './img'
            mask_path = os.path.join(mask_dir, f"{base_name}.png")
            mask_path_out = './msk'
            
            if os.path.exists(aug_img_path):
                shutil.copy(img_path, img_path_out)
            
            if os.path.exists(mask_path):
                shutil.copy(mask_path, mask_path_out)

relation_folder = '../data/relations'
image_dir = '../data/trainval/VOCdevkit/VOC2012/JPEGImages'
mask_dir = '../data/trainval/VOCdevkit/VOC2012/SegmentationClass'
augmented_dir = '../data/trainval/VOCdevkit/VOC2012/AugmentedImages'

# os.mkdir('img')
# os.mkdir('msk')

copy_images_and_masks(relation_folder, image_dir, mask_dir, augmented_dir)


Loading augmented images and masks: 100%|██████████| 47/47 [00:00<00:00, 175.20it/s]


In [ ]:
import os
import random
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import pytorch_lightning as pl
from torch import nn
from torchmetrics import JaccardIndex
from torchmetrics.segmentation import MeanIoU
import numpy as np
import torchvision.models.segmentation as seg_models
import torch.nn.functional as F
import wandb

wandb.init(project="segmentation_project", entity="melytanulo-buvarok")

class VOCDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        self.images = [f for f in sorted(os.listdir(image_dir)) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png').replace('.jpeg', '.png'))
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
         
        if self.transform:
            image = self.transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        mask = np.array(mask, dtype=np.int64)
        return image, torch.from_numpy(mask)

def create_data_loaders(image_dir, mask_dir, batch_size=8, val_split=0.2, num_workers=4):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])

    mask_transform = transforms.Compose([
        transforms.Resize((256, 256), interpolation=Image.NEAREST)
    ])

    full_dataset = VOCDataset(image_dir, mask_dir, transform=transform, mask_transform=mask_transform)
    
    total_size = len(full_dataset)
    val_size = int(val_split * total_size)
    train_size = total_size - val_size

    train_dataset, val_dataset = random_split(
        full_dataset, 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Total dataset size: {total_size}")
    print(f"Training set size: {train_size}")
    print(f"Validation set size: {val_size}")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    return train_loader, val_loader

class UNet(pl.LightningModule):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.pre_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.model = seg_models.fcn_resnet50(pretrained=False, num_classes=num_classes)
        self.post_conv = nn.Conv2d(in_channels=num_classes, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        self.jaccard = JaccardIndex(task='multiclass', num_classes=num_classes, ignore_index=255)
        self.mean_iou = MeanIoU(num_classes=num_classes) # no ignore_index :(

    def forward(self, x):
        x = self.pre_conv(x)
        x = self.model(x)['out']
        x = self.post_conv(x)
        return x
    
    def training_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, masks, ignore_index=255)
        self.log("train_loss", loss)
        
        # Log to WandB
        wandb.log({"train_loss": loss})
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, masks, ignore_index=255)
        preds = outputs.argmax(dim=1)
        
        jaccard = self.jaccard(preds, masks)
        mean_iou = self.mean_iou(preds, masks)
        
        self.log("val_loss", loss)
        self.log("val_jaccard", jaccard)
        self.log("val_mean_iou", mean_iou)
        
        # Log to WandB
        wandb.log({"val_loss": loss, "val_jaccard": jaccard, "val_mean_iou": mean_iou})

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

# Initialize model and trainer
model = UNet(num_classes=21)
trainer = pl.Trainer(max_epochs=3, accelerator='auto', logger=pl.loggers.WandbLogger())
train_loader, val_loader = create_data_loaders('./img', './msk')
trainer.fit(model, train_loader, val_loader)

# Run evaluation on the validation set
def evaluate_model(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    jaccard_meter = JaccardIndex(task='multiclass', num_classes=21, ignore_index=255)
    mean_iou_meter = MeanIoU(num_classes=21)
    val_loss_meter = 0
    count = 0

    with torch.no_grad():
        for batch in val_loader:
            images, masks = batch
            outputs = model(images)
            loss = F.cross_entropy(outputs, masks, ignore_index=255)
            preds = outputs.argmax(dim=1)

            val_loss_meter += loss.item()
            jaccard_meter(preds, masks)
            mean_iou_meter(preds, masks)
            count += 1
    
    avg_val_loss = val_loss_meter / count
    avg_jaccard = jaccard_meter.compute()
    avg_mean_iou = mean_iou_meter.compute()

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Jaccard Index: {avg_jaccard:.4f}")
    print(f"Validation Mean IoU: {avg_mean_iou:.4f}")

# Call the evaluation method
evaluate_model(model, val_loader)

# Save the model
model_path = 'unet_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Finish the W&B run
wandb.finish()


c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\pytorch_lightning\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name     | Type                   | Params | Mode 
-------

Total dataset size: 2391
Training set size: 1913
Validation set size: 478
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


RuntimeError: DataLoader worker (pid(s) 14256, 30508, 25328, 25188) exited unexpectedly